In [1]:
import pickle
import os
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Loading in ...

In [2]:
os.chdir('Data')
pkl_file = open('isochrones.pkl', 'rb')
stacked_isochrones = pickle.load(pkl_file)

pkl_file = open('columns.pkl', 'rb')
x_columns = pickle.load(pkl_file)

pkl_file = open('x_values.pkl', 'rb')
x_values = pickle.load(pkl_file)

pkl_file = open('isoc_cols.pkl', 'rb')
isoc_columns = pickle.load(pkl_file)

x_input=pd.read_csv('x_input')
x_input_err=pd.read_csv('x_input_err')

os.chdir('..')

In [4]:
stacked_isochrones[0]

array([[  3.60605376,   3.88205228,   1.97221484, ...,          nan,
                 nan,          nan],
       [  3.12640321,   0.69069069,   5.        , ...,          nan,
                 nan,          nan],
       [  5.        ,  -4.        ,   0.        , ...,          nan,
                 nan,          nan],
       ...,
       [-18.62651499,  -0.09983469,  -0.03972292, ...,          nan,
                 nan,          nan],
       [ -0.04062704,  24.6141491 , -20.90347589, ...,          nan,
                 nan,          nan],
       [-22.83590309, -22.51460167,   1.46087523, ...,          nan,
                 nan,          nan]])

In [81]:
import torch

In [82]:
def find_nearest(array, value):
    #array = np.asarray(array)
    idx = (torch.abs(array - value)).argmin()
    return array[idx],idx

def isochrone_selector(feh,age):
    '''if(feh<-4 or feh>0.5):
        raise NotImplementedError
    if(age<5 or age>10.3):
        raise NotImplementedError
    else:'''
    logagegrid = torch.tensor(np.linspace(5,10.3,105))
    fehgrid = torch.tensor(np.linspace(-4,0.5,90))
    feh,feh_idx=find_nearest(fehgrid,feh)
    age,age_idx=find_nearest(logagegrid,age)

    return feh_idx*len(logagegrid)+age_idx

In [83]:
def column_index(name):
    if name in x_columns:
        return np.where(np.array(x_columns)==name)[0][0]
    else:
        return np.where(np.array(isoc_columns)==name)[0][0] +len(x_columns)

# Loaded.

In [84]:
x_values[0,0,:]

array([282.60587141, 282.60587141, 282.60587141, 282.60587141,
       282.60587141, 282.60587141, 282.60587141, 282.60587141,
       282.60587141, 282.60587141, 282.60587141, 282.60587141,
       282.60587141, 282.60587141, 282.60587141, 282.60587141,
       282.60587141, 282.60587141, 282.60587141, 282.60587141,
       282.60587141, 282.60587141, 282.60587141, 282.60587141,
       282.60587141, 282.60587141, 282.60587141, 282.60587141,
       282.60587141, 282.60587141, 282.60587141, 282.60587141,
       282.60587141, 282.60587141, 282.60587141, 282.60587141,
       282.60587141, 282.60587141, 282.60587141, 282.60587141,
       282.60587141, 282.60587141, 282.60587141, 282.60587141,
       282.60587141, 282.60587141, 282.60587141, 282.60587141,
       282.60587141, 282.60587141, 282.60587141, 282.60587141,
       282.60587141, 282.60587141, 282.60587141, 282.60587141,
       282.60587141, 282.60587141, 282.60587141, 282.60587141,
       282.60587141, 282.60587141, 282.60587141, 282.60

In [85]:
isoc_columns

array(['logg', 'logteff', 'logl', 'mass', 'logage', 'feh', 'phase',
       'Gaia_RP_EDR3', 'Gaia_BP_EDR3', 'Gaia_G_EDR3', 'BPRP', 'p_slopes',
       'slopes', 'low_c', 'high_c'], dtype='<U12')

# Data

The data we have loaded in is as follows:

1. x_values is a Numpy array of size (n_samples,n_features,longest_isochrone_tang_length). Each sample has n_features which are copied into the 3rd axis the same number of times as the longest isochrone is.
2. Stacked_isochrones is a Numpy array of size (n_isochrones,n_features,largest_tangent_numb_size). Each isochrone will have a certain number of slopes and p_slopes depending on the isochrone. These values extend out into the third axis, however they are padded with NaN values.
3. x_input and err are easier access versions, used for input

In [86]:
stacked_isochrones[0][column_index('p_slopes')-len(x_columns)]


array([-1.73925433e+01, -1.75523825e+01, -1.74907631e+01, -1.59839254e-01,
        7.07219486e-02, -1.41398819e+01, -1.76695745e+01, -1.74794589e+01,
        1.99925974e+00,  4.14005443e+00,  9.40709167e+00,  1.93193193e+00,
        5.00000000e+00, -4.00000000e+00,  0.00000000e+00, -1.75908508e+01,
       -1.77373116e+01, -1.76799325e+01, -1.46460821e-01,  6.14792768e-02,
       -1.62656435e+01, -1.78577022e+01, -1.76709282e+01,  1.97464221e+00,
        4.12802174e+00,  9.45416162e+00,  1.95195195e+00,  5.00000000e+00,
       -4.00000000e+00,  0.00000000e+00, -1.77842444e+01, -1.79192657e+01,
       -1.78660032e+01, -1.35021331e-01,  6.15102157e-02, -1.62574621e+01,
       -1.80437757e+01, -1.78576980e+01,  1.95036799e+00,  4.11602791e+00,
        9.50104221e+00,  1.97197197e+00,  5.00000000e+00, -4.00000000e+00,
        0.00000000e+00, -1.79769199e+01, -1.81005387e+01, -1.80513795e+01,
       -1.23618792e-01,  5.92113304e-02, -1.68886595e+01, -1.82252834e+01,
       -1.80440599e+01,  

# VAESTAR

In [87]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [88]:
device='cpu'#torch.device("mps")
torch.backends.mps.is_available()

sample_size=32

In [89]:
x_input=torch.tensor(x_input.values[:,1:6],requires_grad=True)
x_input=x_input.reshape((x_input.shape)+(1,))
x_input_err=torch.tensor(x_input_err.values[:,1:],requires_grad=True)
x_input_err=x_input_err.reshape((x_input_err.shape)+(1,))

In [90]:
x_max=torch.max(x_input,0)[0]
x_min=x_input.min(axis=0)[0]

Transform values

In [91]:
x_input=(x_input-x_min)/(x_max-x_min)

In [92]:
x_input_err=x_input_err/(x_max-x_min)**2

In [93]:
x_input=x_input.repeat(1,1,sample_size)
x_input_err=x_input_err.repeat(1,1,sample_size)

# Data preparation over
1. x_input is the input for the encoder and x_input_err is the error input
2. x_values are the inputs for the decoder and stacked_isochrones are too.

# VaeStar

In [94]:

class encoder(nn.Module): #q(z|x)
    def __init__(self,input_dim,hidden_dims,z_dim):
        super().__init__()
        # Shapes
        self.sample_size=32
        self.input_dim=1
        self.n_layers=2
        self.lstm_hidden_dim=5

        self.z_dim=z_dim

        self.MV_N=torch.distributions.MultivariateNormal(torch.tensor([0.0 for i in range(self.sample_size)]),torch.eye(self.sample_size))

        # Model Definition
        self.sigmoid=nn.Sigmoid()
        #the shape will be (batch_size,sequencelength=1,input_dim=1)
        self.dist_lstm=nn.LSTM(self.input_dim,self.lstm_hidden_dim,self.n_layers,batch_first=True)
        self.lstm_dense=nn.Linear(in_features=self.sample_size*self.lstm_hidden_dim,out_features=hidden_dims[1])
        self.lstm_activation=nn.Tanh()

        self.input_dense=nn.Linear(in_features=input_dim,out_features=hidden_dims[0])
        self.hidden_dense=nn.Linear(in_features=hidden_dims[0],out_features=hidden_dims[1])
        self.input_activation=nn.ReLU()
        self.hidden_activation=nn.ReLU()

        self.concat_dense=nn.Linear(in_features=hidden_dims[1]*2,out_features=z_dim*2)
        self.z_activation=nn.ReLU() #this will mean that extinction cant be negative (this is actually a part of the prior i suppose), could also just do linear

        self.N=torch.distributions.Normal(0,1) #prior on extinction

        
    def forward(self, x,x_err):
        # adjust the data
        
        eps=self.MV_N.sample()
        x=x+eps.to(device)*x_err
        
        x_p=x[:,2,:] #very specific to form of data
        x_np=x[:,[0,1,3,4],:]
        x_np=x_np.reshape((x_np.shape[0]*x_np.shape[2],x_np.shape[1])) #stacking and will average later
        #x_np[:,3]=x_np[:,3]+5*torch.log10(x_p.reshape((x_p.shape[0]*x_p.shape[1]))/1000)+5 #absolute magnitude
        # absolute magnitude is messing everything up - need to change to make sure there are no nans., that will mean a prior on the distance.
        
        #Neural Network - Not parallax
        x_np=self.input_activation(self.input_dense(x_np))
        x_np=self.hidden_activation(self.hidden_dense(x_np))
        x_np=torch.mean(x_np.reshape((int(x_np.shape[0]/self.sample_size),x_np.shape[1],int(self.sample_size))),-1)


        # Neural Network LSTM - parallax
        h_0 = torch.zeros(2, x_p.size(0), self.lstm_hidden_dim) #hidden state
        c_0 = torch.zeros(2, x_p.size(0), self.lstm_hidden_dim) #internal state
        # Propagate input through LSTM
        
        x_p, (hn,cn) =self.dist_lstm(x_p.reshape(x_p.shape+(1,)),(h_0,c_0))
        x_p=self.lstm_activation(self.lstm_dense(x_p.reshape(x_p.shape[0],x_p.shape[1]*x_p.shape[2]))) # is this too much magic
        
        #concatenate channež
 
        x=torch.concat([x_np,x_p],axis=1)
        
        output=self.concat_dense(x)


        #sample a z value now
        z_mu=output[:,:self.z_dim]
        z_mu[:,:2]=2*self.sigmoid(z_mu[:,:2])
        z_mu[:,2]=2.25*self.sigmoid(z_mu[:,2])-1.75
        z_mu[:,3]=2.65*self.sigmoid(z_mu[:,2])+7.65


        z_sigma=torch.exp(self.sigmoid(output[:,self.z_dim:]))
        z=z_mu+z_sigma*self.N.sample(z_mu.shape)
        # note we have no final activations
        
        z_sigma=torch.stack(list(map(lambda n: torch.diag(z_sigma[n]),range(len(z_sigma)))))
        q=torch.distributions.multivariate_normal.MultivariateNormal(loc=z_mu,covariance_matrix=z_sigma**2)
        p=torch.distributions.multivariate_normal.MultivariateNormal(torch.tensor([0,0,-2,7.5]),torch.diag(torch.tensor([1,1,1.5,1.5])))

        z_ext=z[:,:2]
        z_feh=z[:,2]
        z_age=z[:,3]
        
        # variances need to be done
        
        

        return z, z_ext,z_feh,z_age, torch.distributions.kl_divergence(q,p)



In [95]:

class decoder(nn.Module):
    def __init__(self):
        super(decoder,self).__init__()
        
    def forward(self,z,all_isochrones,x_values):
        
        z_ext=z[:,:2]
        z_feh=z[:,2]
        z_age=z[:,3]

        log_prob=[]
        log_l=0.0
        
        for i in range(len(z)):

            isochrone=torch.cat([x_values[i],all_isochrones[isochrone_selector(z_feh[i],z_age[i])]],dim=0)

            isochrone=isochrone.reshape((1,)+isochrone.shape)
            print(z_ext.shape)
            print(isochrone[:,column_index('G'),:].shape)
            truth_1=(isochrone[:,column_index('G'),:]+z_ext[i,0]-(isochrone[:,column_index('bp_rp'),:]+z_ext[i,1])*isochrone[:,column_index('p_slopes'),:]<=isochrone[:,column_index('high_c'),:]) #box selection
            truth_1=truth_1.reshape(truth_1.shape[0],1,truth_1.shape[1])
            truth_2=(isochrone[:,column_index('low_c'),:]<=isochrone[:,column_index('G'),:]+z_ext[i,0]-(isochrone[:,column_index('bp_rp'),:]+z_ext[i,1])*isochrone[:,column_index('p_slopes'),:])
            truth_2=truth_2.reshape(truth_2.shape[0],1,truth_2.shape[1])
            truth=truth_1*truth_2
            # ^box selection

            # projection onto the nearest line
            x=((1/torch.sqrt(1+isochrone[:,column_index('slopes'),:]**2))*(isochrone[:,column_index('G'),:]+z_ext[i,0]-(isochrone[:,column_index('bp_rp'),:]+z_ext[i,1])*isochrone[:,column_index('slopes'),:]-isochrone[:,column_index('Gaia_G_EDR3'),:] + isochrone[:,column_index('slopes'),:]*isochrone[:,column_index('BPRP'),:]))
            # taking the minimum
            idx=torch.argmin(torch.abs(x/truth.reshape(x.shape)).nan_to_num(nan=torch.inf),1)
            x=x.gather(1,idx.view(-1,1))
            #error needs to be corrected for absolute magnitude 
            x_err=(1/(1+isochrone[:,column_index('slopes'),:]**2))*isochrone[:,column_index('phot_g_mean_mag_error'),:]**2+(isochrone[:,column_index('slopes'),:]*isochrone[:,column_index('bp_rp_error'),:])**2
            x_err=x_err.gather(1,idx.view(-1,1))
            isochrone=torch.cat((isochrone,x.reshape(x.shape[0],1,x.shape[1]).repeat(1,1,(isochrone).shape[-1]),x_err.reshape(x_err.shape[0],1,x_err.shape[1]).repeat(1,1,(isochrone).shape[-1])),1)
                            
            dist=torch.distributions.multivariate_normal.MultivariateNormal(torch.zeros_like(x),torch.eye(len(x))+torch.diag(x_err**2))
            
            
            
            try:# serious issues here
                
                log_l+=dist.log_prob(x)
                log_prob.append(dist.log_prob(x))

            except:
                
                log_l+=0.0
                log_prob.append(0.0)

        
        return log_l,log_prob,z
        


class VaeStar(nn.Module):
    def __init__(self,input_dim,hidden_dims,z_dim):
        super(VaeStar, self).__init__()
        self.encoder=encoder(input_dim,hidden_dims,z_dim)
        self.decoder=decoder()
    
    def forward(self,x_input,x_input_err, x_values, all_isochrones):
        
        z, z_ext,z_feh,z_age, kl=self.encoder(x_input,x_input_err)
        log_l,log_prob,z=self.decoder(z,all_isochrones,x_values)
        
        return kl, log_l,log_prob,z






In [96]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

In [97]:
x_input.shape[1]

5

In [98]:
lr=1e-3
model=VaeStar(input_dim=x_input.shape[1]-1,hidden_dims=[10,10],z_dim=4)
optimizer=torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
model.to(device)

VaeStar(
  (encoder): encoder(
    (sigmoid): Sigmoid()
    (dist_lstm): LSTM(1, 5, num_layers=2, batch_first=True)
    (lstm_dense): Linear(in_features=160, out_features=10, bias=True)
    (lstm_activation): Tanh()
    (input_dense): Linear(in_features=4, out_features=10, bias=True)
    (hidden_dense): Linear(in_features=10, out_features=10, bias=True)
    (input_activation): ReLU()
    (hidden_activation): ReLU()
    (concat_dense): Linear(in_features=20, out_features=8, bias=True)
    (z_activation): ReLU()
  )
  (decoder): decoder()
)

In [99]:
batch_size = 100#draw_size #need to make sure everything adds up
lr = 1e-3
epochs = 50

In [100]:
x_cat=torch.cat([x_input,x_input_err],dim=1)

In [101]:
x_values=torch.tensor(x_values)
stacked_isochrones=torch.tensor(stacked_isochrones)

In [102]:
from torch.utils.data import DataLoader
x_cat=DataLoader(x_cat.float(),batch_size=batch_size,drop_last=True)

In [103]:
x_values[0]

tensor([[ 2.8261e+02,  2.8261e+02,  2.8261e+02,  ...,  2.8261e+02,
          2.8261e+02,  2.8261e+02],
        [-6.2860e+00, -6.2860e+00, -6.2860e+00,  ..., -6.2860e+00,
         -6.2860e+00, -6.2860e+00],
        [ 5.0739e-01,  5.0739e-01,  5.0739e-01,  ...,  5.0739e-01,
          5.0739e-01,  5.0739e-01],
        ...,
        [ 4.9026e-03,  4.9026e-03,  4.9026e-03,  ...,  4.9026e-03,
          4.9026e-03,  4.9026e-03],
        [ 7.6935e-03,  7.6935e-03,  7.6935e-03,  ...,  7.6935e-03,
          7.6935e-03,  7.6935e-03],
        [ 4.9308e+00,  4.9308e+00,  4.9308e+00,  ...,  4.9308e+00,
          4.9308e+00,  4.9308e+00]], dtype=torch.float64)

In [104]:
model.train()
for epoch in range(epochs):
    overall_loss=0.0
    for batch_idx,x in enumerate(x_cat):
        x,x_err=torch.split(x,split_size_or_sections=int((x.shape[1]/2)),dim=1)
        x=x.view(batch_size,x.shape[1],x.shape[2])
        x_err=x_err.view(batch_size,x_err.shape[1],x_err.shape[2])
        x=x.to(device)
        x_err=x_err.to(device)
        
        
        
        optimizer.zero_grad()
        
        kl, log_l, log_prob, z = model(x,x_err,x_values,stacked_isochrones)

        loss=kl.mean()-log_l
        overall_loss+=loss.item()
        loss.backward(retain_graph=True)
        optimizer.step()
    print("\tEpoch", epoch + 1, "complete!", "\tAverage Loss: ", overall_loss / ((batch_idx+1)*batch_size))
    print("Overall Loss: ", overall_loss)


torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1, 457])
torch.Size([100, 2])
torch.Size([1

ValueError: Expected parameter loc (Tensor of shape (100, 4)) of distribution MultivariateNormal(loc: torch.Size([100, 4]), covariance_matrix: torch.Size([100, 4, 4])) to satisfy the constraint IndependentConstraint(Real(), 1), but found invalid values:
tensor([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, na

In [ ]:
# this makes no sense it has no way of knowing the x in the output space need to check the indices

tensor([ 0.9616, -2.2709, -2.1299, -3.6078, -1.6145,  0.1445,  1.0100, -0.4319,
        -0.9180, -1.0836, -0.9469, -2.0988, -0.2342, -1.9720, -1.5815, -3.4240,
        -2.7625, -0.8353, -2.4097, -0.5739, -1.2914, -1.4979, -2.2890, -3.5471,
        -1.8806,  0.2739, -1.8944, -1.1437, -2.3654, -2.1998, -0.9713, -0.3595,
        -2.7996, -1.5337, -0.3985,  0.1328, -1.2754, -3.1301,  0.3384, -1.5372,
        -3.9976, -3.0134, -2.4749,  0.2650, -2.5529, -1.4431, -0.5315, -5.2530,
        -3.2311, -1.8640, -0.0294, -3.0989, -1.2733, -1.5375, -3.5301, -1.4240,
        -0.7318,  0.4610, -3.2573, -1.9152, -2.3645, -0.0736, -0.8121, -2.0180,
        -1.3153, -1.8200, -1.8575, -1.7040, -1.8140, -3.0763, -1.9474,  0.8322,
        -3.3599, -2.1923, -3.0234, -4.1302, -0.8596, -2.3136, -1.8117, -2.5437,
        -3.5837, -0.6596, -1.0971, -1.0426, -2.1915, -2.2607, -2.2000, -2.1488,
        -2.7520, -2.3063,  0.8070, -2.8988, -0.1996, -1.8477, -2.6091, -1.1129,
         0.3856, -2.4116, -2.7115, -2.16